In [106]:
import DataLoader
import importlib
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import CoverageHelper
importlib.reload(DataLoader)
%matplotlib inline

Number of apps: 
412


In [80]:
test_graph_df = DataLoader.load_csv_test_graphs()
# stat_graph_df = DataLoader.load_csv_stat_graph("")
# stat_trans_graph_df = DataLoader.load_csv_stat_graph("-transitive")
stat_trans_act_graph_df = DataLoader.load_csv_stat_graph("-act-transitive")

Total entries:
2088


Total entries:
41163


In [81]:
test_graph_df.head()


,source,target,action,widget_id,widget,strategy,app
0,com.xgvrk.kxkid87795.AdActivity,pawelz.Apps.Guns.Shoot.Sounds.GunsShotSoundsAc...,back,NaN,NaN,systematic,pawelz.Apps.Guns.Shoot.Sounds
1,com.xgvrk.kxkid87795.AdActivity,pawelz.Apps.Guns.Shoot.Sounds.GunsShotSoundsAc...,click,2.131296e+09,android.widget.Button,systematic,pawelz.Apps.Guns.Shoot.Sounds
2,com.xgvrk.kxkid87795.AdActivity,pawelz.Apps.Guns.Shoot.Sounds.GunsShotSoundsAc...,click,2.131296e+09,android.widget.Button,systematic,pawelz.Apps.Guns.Shoot.Sounds
3,pawelz.Apps.Guns.Shoot.Sounds.SplashScreen,pawelz.Apps.Guns.Shoot.Sounds.GunsShotSoundsAc...,back,NaN,NaN,systematic,pawelz.Apps.Guns.Shoot.Sounds
4,com.xgvrk.kxkid87795.AdActivity,pawelz.Apps.Guns.Shoot.Sounds.GunsShotSoundsAc...,clickText,-2.000000e+00,android.widget.TextView,random,pawelz.Apps.Guns.Shoot.Sounds


In [82]:
test_graph_df[test_graph_df.strategy == "random"].app.nunique()
p = test_graph_df[~test_graph_df.app.isin(stat_trans_act_graph_df.app)]
p.app.nunique()
# stat_trans_act_graph_df[stat_trans_act_graph_df.app == "com.littlebithackers.babyns"]

36

In [83]:
lol = stat_trans_act_graph_df[["app", "method", "widget"]]
lol[~lol.widget.notnull()].groupby(["app", "method"]).widget.value_counts(dropna=False).to_frame(
    name="widget_count").sort_values(by=["widget_count"], ascending=False).head()


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th></th>
      <th>widget_count</th>
    </tr>
    <tr>
      <th>app</th>
      <th>method</th>
      <th>widget</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>com.yinzcam.nfl.packers</th>
      <th>onCreate</th>
      <th>NaN</th>
      <td>1769</td>
    </tr>
    <tr>
      <th>com.veclink.movnow.healthy_q2</th>
      <th>startActivity</th>
      <th>NaN</th>
      <td>741</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">com.qxmd.readbyqxmd</th>
      <th>onResume</th>
      <th>NaN</th>
      <td>614</td>
    </tr>
    <tr>
      <th>onCreate</th>
      <th>NaN</th>
      <td>596</td>
    </tr>
    <tr>
      <th>onPause</th>
      <th>NaN</th>
      <td>527</td>
    </tr>
  </tbody>
</table>
</div>

In [84]:
# stat_trans_graph_df.head()

In [85]:
#difference
# pd.concat([stat_trans_graph_df,stat_trans_act_graph_df]).drop_duplicates(keep=False)


In [86]:
unique_widgets_per_app = test_graph_df.groupby(["strategy", "app"]).widget.value_counts(dropna=False).to_frame(
    name="widget_count")

unique_widgets_per_app.head()

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th></th>
      <th>widget_count</th>
    </tr>
    <tr>
      <th>strategy</th>
      <th>app</th>
      <th>widget</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="5" valign="top">random</th>
      <th rowspan="2" valign="top">alex.map.maps</th>
      <th>NaN</th>
      <td>3</td>
    </tr>
    <tr>
      <th>android.widget.Button</th>
      <td>2</td>
    </tr>
    <tr>
      <th>androidlab.carlog</th>
      <th>NaN</th>
      <td>1</td>
    </tr>
    <tr>
      <th>appinventor.ai_agiapplications.CountYourSteps</th>
      <th>android.widget.Button</th>
      <td>1</td>
    </tr>
    <tr>
      <th>asp.aniemo.pack6.ext</th>
      <th>NaN</th>
      <td>1</td>
    </tr>
  </tbody>
</table>
</div>

In [87]:
unique_widgets_per_app['percentage'] = unique_widgets_per_app.groupby(level=0).apply(lambda x: 100 * x / x.sum())
unique_widgets_per_app

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th></th>
      <th></th>
      <th>widget_count</th>
      <th>percentage</th>
    </tr>
    <tr>
      <th>strategy</th>
      <th>app</th>
      <th>widget</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th rowspan="30" valign="top">random</th>
      <th rowspan="2" valign="top">alex.map.maps</th>
      <th>NaN</th>
      <td>3</td>
      <td>0.247934</td>
    </tr>
    <tr>
      <th>android.widget.Button</th>
      <td>2</td>
      <td>0.165289</td>
    </tr>
    <tr>
      <th>androidlab.carlog</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th>appinventor.ai_agiapplications.CountYourSteps</th>
      <th>android.widget.Button</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th>asp.aniemo.pack6.ext</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th>bestfreelivewallpapers.funny_photo_editor</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th>bestfreelivewallpapers.love_photo_frames_hd</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th>biz.andxor.hearingaid</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th>block.app.wars</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">br.com.blackmountain.mylook</th>
      <th>android.support.v7.widget.CardView</th>
      <td>2</td>
      <td>0.165289</td>
    </tr>
    <tr>
      <th>NaN</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th>android.widget.Button</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th>br.com.geochenapps.tonguetwister</th>
      <th>android.widget.ImageButton</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th>br.org.altitudesl</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th rowspan="2" valign="top">brdata.cms.base.foodbazaar</th>
      <th>android.widget.ImageSwitcher</th>
      <td>2</td>
      <td>0.165289</td>
    </tr>
    <tr>
      <th>NaN</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th rowspan="3" valign="top">bth.studio.quickcharge</th>
      <th>android.widget.FrameLayout</th>
      <td>14</td>
      <td>1.157025</td>
    </tr>
    <tr>
      <th>android.widget.ToggleButton</th>
      <td>6</td>
      <td>0.495868</td>
    </tr>
    <tr>
      <th>NaN</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th rowspan="2" valign="top">carrioncastillo.invitaciones.happy</th>
      <th>NaN</th>
      <td>2</td>
      <td>0.165289</td>
    </tr>
    <tr>
      <th>android.widget.Button</th>
      <td>2</td>
      <td>0.165289</td>
    </tr>
    <tr>
      <th>church.itvessel.com.tarm</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th>co.appzone.www.eye_makeup</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th>co.instavr.iostest</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th>co.mobius.vrcinema</th>
      <th>android.widget.ImageButton</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th>coloring.book.color</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th>com.BillDirkes.QuickEM</th>
      <th>NaN</th>
      <td>1</td>
      <td>0.082645</td>
    </tr>
    <tr>
      <th>com.CarsLiveWallpaperHQ</th>
      <th>NaN</th>
      <td>1</td>
   

In [88]:
number_of_unique_screens_per_app = test_graph_df[['strategy', 'app', 'source', 'target', 'widget']].melt(
    id_vars=['strategy', 'app', 'widget'])
number_of_unique_screens_per_app

,strategy,app,widget,variable,value
0,systematic,pawelz.Apps.Guns.Shoot.Sounds,NaN,source,com.xgvrk.kxkid87795.AdActivity
1,systematic,pawelz.Apps.Guns.Shoot.Sounds,android.widget.Button,source,com.xgvrk.kxkid87795.AdActivity
2,systematic,pawelz.Apps.Guns.Shoot.Sounds,android.widget.Button,source,com.xgvrk.kxkid87795.AdActivity
3,systematic,pawelz.Apps.Guns.Shoot.Sounds,NaN,source,pawelz.Apps.Guns.Shoot.Sounds.SplashScreen
4,random,pawelz.Apps.Guns.Shoot.Sounds,android.widget.TextView,source,com.xgvrk.kxkid87795.AdActivity
5,systematic,br.org.altitudesl,NaN,source,br.org.altitudesl.MainActivity
6,random,br.org.altitudesl,NaN,source,br.org.altitudesl.MainActivity
7,systematic,block.app.wars,NaN,source,com.prime31.UnityPlayerNativeActivity
8,random,block.app.wars,NaN,source,com.prime31.UnityPlayerNativeActivity
9,systematic,com.gradientkeyboard.galaxyemoji,NaN,source,com.gradientkeyboard.galaxyemoji.MainActivity2


Edges from

In [89]:
systematic_test_graph = test_graph_df[test_graph_df.strategy == "systematic"]
systematic_test_graph.head()

,source,target,action,widget_id,widget,strategy,app
0,com.xgvrk.kxkid87795.AdActivity,pawelz.Apps.Guns.Shoot.Sounds.GunsShotSoundsAc...,back,NaN,NaN,systematic,pawelz.Apps.Guns.Shoot.Sounds
1,com.xgvrk.kxkid87795.AdActivity,pawelz.Apps.Guns.Shoot.Sounds.GunsShotSoundsAc...,click,2.131296e+09,android.widget.Button,systematic,pawelz.Apps.Guns.Shoot.Sounds
2,com.xgvrk.kxkid87795.AdActivity,pawelz.Apps.Guns.Shoot.Sounds.GunsShotSoundsAc...,click,2.131296e+09,android.widget.Button,systematic,pawelz.Apps.Guns.Shoot.Sounds
3,pawelz.Apps.Guns.Shoot.Sounds.SplashScreen,pawelz.Apps.Guns.Shoot.Sounds.GunsShotSoundsAc...,back,NaN,NaN,systematic,pawelz.Apps.Guns.Shoot.Sounds
5,br.org.altitudesl.MainActivity,NaN,NaN,NaN,NaN,systematic,br.org.altitudesl


In [90]:
random_test_graph = test_graph_df[(test_graph_df.strategy == "random")]
random_test_graph.head()


,source,target,action,widget_id,widget,strategy,app
4,com.xgvrk.kxkid87795.AdActivity,pawelz.Apps.Guns.Shoot.Sounds.GunsShotSoundsAc...,clickText,-2.0,android.widget.TextView,random,pawelz.Apps.Guns.Shoot.Sounds
6,br.org.altitudesl.MainActivity,NaN,NaN,NaN,NaN,random,br.org.altitudesl
8,com.prime31.UnityPlayerNativeActivity,NaN,NaN,NaN,NaN,random,block.app.wars
10,com.gradientkeyboard.galaxyemoji.MainActivity2,NaN,NaN,NaN,NaN,random,com.gradientkeyboard.galaxyemoji
12,com.vpar.android.ui.dispatch.DispatchActivity,NaN,NaN,NaN,NaN,random,com.vpar.android


Systematic:

In [126]:
direct_edges_not_found = CoverageHelper.diff_screens(stat_trans_act_graph_df, systematic_test_graph)
direct_edges_not_found = direct_edges_not_found[~direct_edges_not_found.target.isnull()]
# unique_widgets_per_app = CoverageHelper.widget_counter(direct_edges_not_found)
# unique_widgets_per_app.head()

unique_widgets_per_app = direct_edges_not_found.widget.value_counts(dropna=False).to_frame(
    name="widget_count")
unique_widgets_per_app.head()

,widget_count
NaN,1222
android.view.View,266
android.view.MenuItem,265
android.widget.Button,199
android.widget.ImageView,198


widgets from missed edges from systematic test graphs

In [128]:
unique_widgets_per_app = unique_widgets_per_app[~unique_widgets_per_app.index.isnull()]
unique_widgets_per_app['percentage'] = unique_widgets_per_app.apply(lambda x: 100 * x / x.sum())
unique_widgets_per_app.sort_values(by=["widget_count"], ascending=False).iloc[:10]

,widget_count,percentage
android.view.View,266,11.949686
android.view.MenuItem,265,11.904762
android.widget.Button,199,8.939802
android.widget.ImageView,198,8.894879
android.widget.ListView,186,8.355795
android.widget.TextView,106,4.761905
android.app.AlertDialog$Builder,105,4.716981
android.widget.RelativeLayout,65,2.920036
android.widget.PopupMenu,57,2.560647
android.view.ViewGroup,47,2.111411


Random:

In [129]:
direct_edges_not_found = CoverageHelper.diff_screens(stat_trans_act_graph_df, random_test_graph)
direct_edges_not_found = direct_edges_not_found[~direct_edges_not_found.target.isnull()]
unique_widgets_per_app = direct_edges_not_found.widget.value_counts(dropna=False).to_frame(
    name="widget_count")
unique_widgets_per_app.head()

,widget_count
NaN,1319
android.view.MenuItem,302
android.view.View,280
android.widget.ListView,206
android.widget.ImageView,193


In [130]:
unique_widgets_per_app = unique_widgets_per_app[~unique_widgets_per_app.index.isnull()]
unique_widgets_per_app['percentage'] = unique_widgets_per_app.apply(lambda x: 100 * x / x.sum())
unique_widgets_per_app.sort_values(by=["widget_count"], ascending=False).iloc[:10]

,widget_count,percentage
android.view.MenuItem,302,10.758817
android.view.View,280,9.975062
android.widget.ListView,206,7.338796
android.widget.ImageView,193,6.875668
android.widget.Button,191,6.804418
android.widget.TextView,127,4.524403
android.app.AlertDialog$Builder,113,4.025650
android.widget.RelativeLayout,76,2.707517
android.widget.PopupMenu,57,2.030638
android.widget.LinearLayout,53,1.888137


In [140]:
direct_edges_not_found = CoverageHelper.diff_screens(stat_trans_act_graph_df, random_test_graph)
direct_edges_not_found = direct_edges_not_found[~direct_edges_not_found.target.isnull()]
direct_edges_not_found = direct_edges_not_found[direct_edges_not_found.widget.isnull()]
unique_widgets_per_app = direct_edges_not_found.groupby(["method"]).widget.value_counts(dropna=False).to_frame(
    name="widget_count")
unique_widgets_per_app.sort_values(by=["widget_count"], ascending=False)

,,widget_count
method,widget,
onCreate,NaN,292
onActivityResult,NaN,123
onResume,NaN,110
a,NaN,69
startActivityForResult,NaN,54
onStart,NaN,38
onNewIntent,NaN,36
onAttach,NaN,35
onPause,NaN,33
